# In contours with sample MPAS

In [ ]:
import sys; sys.path.append("../")

import matplotlib as mpl
import pandas as pd

import tams

In [ ]:
ds = tams.load_example_mpas().isel(time=1)
tb = ds.tb
precip = ds.precip
precip.where(precip > 0).plot(norm=mpl.colors.LogNorm(vmin=1e-5))

In [ ]:
res = tams.identify(tb)
cs235, cs219 = res[0][0], res[1][0]

In [ ]:
tams.data_in_contours(precip, cs219[:5], merge=True)

In [ ]:
cs235

In [ ]:
# 219 K stats inside CEs

# Add data to 219 data (not needed really)
cs219 = tams.data_in_contours(tb, cs219, merge=True)


# Row-wise lookup and apply method

def f(row):
    """Compute stats for 219s associated with a certain 245."""
    return tams.data_in_contours(tb, cs219.iloc[row.inds219].dissolve())

res = cs235.apply(f, axis="columns")  # Series of DataFrames
res.head()
pd.concat(res.to_list()).reset_index(drop=True).rename(columns={"count_tb": "npixels219"})

In [ ]:
# 219 MultiPolygon column method

import geopandas as gpd
from shapely.geometry import MultiPolygon

In [ ]:
%timeit gpd.GeoSeries(cs235.inds219.apply(lambda inds: cs219.iloc[inds][["geometry"]].dissolve().geometry[0]))

In [ ]:
%timeit gpd.GeoSeries(cs235.inds219.apply(lambda inds: MultiPolygon(cs219.geometry.iloc[inds].values)).rename("cs219"))

In [ ]:
tams.data_in_contours(
    precip,
    gpd.GeoDataFrame(crs="EPSG:4326", geometry=cs235.inds219.apply(lambda inds: MultiPolygon(cs219.geometry.iloc[inds].values)).rename("cs219")),
)

In [ ]:
tmp = gpd.GeoSeries(cs235.inds219.apply(lambda inds: MultiPolygon(cs219.geometry.iloc[inds].values)).rename("cs219"))
cs235["cs219"] = tmp
cs235

In [ ]:
cs235.set_geometry("cs219").dissolve().cs219.geometry.values[0]

In [ ]:
len(cs235.set_geometry("cs219").dissolve().cs219.geometry.values[0].geoms)

In [ ]:
for i in cs235.dissolve().cs219.values[0].geoms: print(i)

In [ ]:
cs235.cs219.explode(index_parts=True).reset_index()

In [ ]:
tams.data_in_contours(tb, cs235.set_geometry("cs219", drop=True)).add_suffix("219")
# Without `drop=True` we get `AttributeError: 'DataFrame' object has no attribute 'geometry'`
# (seems like a bug)

In [ ]:
tams.data_in_contours(tb, gpd.GeoDataFrame(geometry=cs235.set_geometry("cs219").geometry))

In [ ]:
# A check to see that the 235s have more count
tams.data_in_contours(tb, cs235, merge=True)

In [ ]:
# 219 K stats inside MCSs